# Loop 4 LB Feedback Analysis

## Key Findings:
- exp_003 (fix_direction) scored LB 70.6473 - exactly matching CV
- CV-LB gap is ZERO - local validation is perfectly calibrated
- Three optimization attempts (bbox3, sa_fast_v2, fix_direction) all failed to improve baseline
- The baseline is at a strong local optimum

## Strategic Pivot: ENSEMBLE APPROACH

The jonathanchan kernel shows the winning strategy:
1. Gather solutions from MULTIPLE sources (15+ in their case)
2. For each N, pick the BEST solution from all sources
3. Combine into final submission

We have 114 snapshots available - let's analyze them systematically.

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from collections import defaultdict
import json

# Tree template for scoring
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

def score_group(xs, ys, degs):
    """Calculate score for a single N configuration."""
    n = len(xs)
    minx, miny = 1e300, 1e300
    maxx, maxy = -1e300, -1e300
    
    for i in range(n):
        r = np.radians(degs[i])
        c, s = np.cos(r), np.sin(r)
        for j in range(15):
            X = c * TX[j] - s * TY[j] + xs[i]
            Y = s * TX[j] + c * TY[j] + ys[i]
            minx = min(minx, X)
            maxx = max(maxx, X)
            miny = min(miny, Y)
            maxy = max(maxy, Y)
    
    side = max(maxx - minx, maxy - miny)
    return side * side / n

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

print("Functions defined")

In [ ]:
# Scan ALL snapshots and compute per-N scores
snapshot_dir = '/home/nonroot/snapshots/santa-2025/'
snapshots = sorted([d for d in os.listdir(snapshot_dir) if os.path.isdir(os.path.join(snapshot_dir, d))])

print(f"Total snapshots: {len(snapshots)}")

# Store per-N scores for each snapshot
all_scores = {}  # {snapshot_id: {n: score}}
all_total_scores = {}  # {snapshot_id: total_score}

for snap in snapshots:
    csv_path = os.path.join(snapshot_dir, snap, 'submission', 'submission.csv')
    if not os.path.exists(csv_path):
        continue
    
    try:
        df = pd.read_csv(csv_path)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            continue
        
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        
        scores = {}
        for n, g in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = strip(g['x'].values)
            ys = strip(g['y'].values)
            ds = strip(g['deg'].values)
            scores[n] = score_group(xs, ys, ds)
        
        if len(scores) == 200:
            all_scores[snap] = scores
            all_total_scores[snap] = sum(scores.values())
    except Exception as e:
        pass

print(f"Valid snapshots with 200 N values: {len(all_scores)}")

In [ ]:
# Sort snapshots by total score
sorted_snaps = sorted(all_total_scores.items(), key=lambda x: x[1])

print("Top 20 snapshots by total score:")
print("="*60)
for snap, score in sorted_snaps[:20]:
    print(f"{snap}: {score:.6f}")

print(f"\nBaseline (21328309254): {all_total_scores.get('21328309254', 'N/A'):.6f}")

In [ ]:
# Build ensemble: for each N, pick the BEST solution
best_per_n = {}  # {n: (best_score, best_snapshot)}

for n in range(1, 201):
    best_score = float('inf')
    best_snap = None
    
    for snap, scores in all_scores.items():
        if scores[n] < best_score:
            best_score = scores[n]
            best_snap = snap
    
    best_per_n[n] = (best_score, best_snap)

# Calculate ensemble total score
ensemble_score = sum(best_per_n[n][0] for n in range(1, 201))
baseline_score = all_total_scores.get('21328309254', 70.647327)

print(f"Ensemble score (best per N): {ensemble_score:.6f}")
print(f"Baseline score: {baseline_score:.6f}")
print(f"Improvement: {baseline_score - ensemble_score:.6f}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {ensemble_score - 68.888293:.6f}")

In [ ]:
# Analyze which snapshots contribute most to the ensemble
contribution_count = defaultdict(int)
contribution_improvement = defaultdict(float)

baseline_scores = all_scores.get('21328309254', {})

for n in range(1, 201):
    best_score, best_snap = best_per_n[n]
    contribution_count[best_snap] += 1
    
    if n in baseline_scores:
        improvement = baseline_scores[n] - best_score
        if improvement > 0:
            contribution_improvement[best_snap] += improvement

print("Top contributing snapshots:")
print("="*60)
for snap, count in sorted(contribution_count.items(), key=lambda x: -x[1])[:15]:
    total_imp = contribution_improvement.get(snap, 0)
    print(f"{snap}: {count:3d} N values, total improvement: {total_imp:.6f}")

In [ ]:
# Show which N values have the most improvement potential
n_improvements = []
for n in range(1, 201):
    best_score, best_snap = best_per_n[n]
    if n in baseline_scores:
        improvement = baseline_scores[n] - best_score
        if improvement > 0:
            n_improvements.append((n, improvement, best_snap))

n_improvements.sort(key=lambda x: -x[1])

print("Top 20 N values with most improvement from ensemble:")
print("="*60)
for n, imp, snap in n_improvements[:20]:
    print(f"N={n:3d}: improvement={imp:.6f} from {snap}")

print(f"\nTotal N values improved: {len(n_improvements)}")
print(f"Total improvement: {sum(x[1] for x in n_improvements):.6f}")

In [ ]:
# KEY INSIGHT: Check if ensemble has overlaps
# We need to validate that the best solutions don't have overlaps

from shapely.geometry import Polygon
from shapely import affinity
from decimal import Decimal, getcontext

getcontext().prec = 25

def make_tree_polygon(cx, cy, deg):
    """Create tree polygon at given position and rotation."""
    trunk_w, trunk_h = 0.15, 0.2
    base_w, mid_w, top_w = 0.7, 0.4, 0.25
    tip_y, t1, t2, base_y = 0.8, 0.5, 0.25, 0.0
    trunk_bottom_y = -trunk_h
    
    coords = [
        (0.0, tip_y),
        (top_w/2, t1), (top_w/4, t1),
        (mid_w/2, t2), (mid_w/4, t2),
        (base_w/2, base_y),
        (trunk_w/2, base_y), (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y), (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, t2), (-mid_w/2, t2),
        (-top_w/4, t1), (-top_w/2, t1),
    ]
    poly = Polygon(coords)
    poly = affinity.rotate(poly, deg, origin=(0, 0))
    poly = affinity.translate(poly, xoff=cx, yoff=cy)
    return poly

def check_overlaps(xs, ys, degs, tolerance=1e-12):
    """Check if any trees overlap."""
    polygons = [make_tree_polygon(xs[i], ys[i], degs[i]) for i in range(len(xs))]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > tolerance:
                    return True, intersection.area
    return False, 0

print("Overlap checking functions defined")

In [ ]:
# Check overlaps for top contributing snapshots
print("Checking overlaps in top contributing snapshots...")
print("="*60)

top_snaps = [snap for snap, _ in sorted(contribution_count.items(), key=lambda x: -x[1])[:10]]

for snap in top_snaps:
    csv_path = os.path.join(snapshot_dir, snap, 'submission', 'submission.csv')
    df = pd.read_csv(csv_path)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    overlap_count = 0
    for n, g in df.groupby('N'):
        xs = strip(g['x'].values)
        ys = strip(g['y'].values)
        ds = strip(g['deg'].values)
        
        has_overlap, area = check_overlaps(xs, ys, ds)
        if has_overlap:
            overlap_count += 1
    
    status = "✓ VALID" if overlap_count == 0 else f"✗ {overlap_count} overlaps"
    print(f"{snap}: {status}")

In [ ]:
# Summary and next steps
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\nTotal valid snapshots: {len(all_scores)}")
print(f"Best single snapshot: {sorted_snaps[0][0]} with score {sorted_snaps[0][1]:.6f}")
print(f"Ensemble score (best per N): {ensemble_score:.6f}")
print(f"Baseline score: {baseline_score:.6f}")
print(f"Improvement from ensemble: {baseline_score - ensemble_score:.6f}")
print(f"\nTarget: 68.888293")
print(f"Gap to target: {ensemble_score - 68.888293:.6f}")

print("\n" + "="*60)
print("NEXT STEPS")
print("="*60)
print("1. Build ensemble submission from valid (non-overlapping) snapshots")
print("2. Apply fractional translation refinement (jonathanchan approach)")
print("3. Run additional SA optimization on ensemble")
print("4. Target specific N values with largest gaps")